In [8]:
import matplotlib.pyplot as plt
import numpy as np
import scipy.integrate as spi
import scipy.interpolate as spip
from sympy import *
from sympy.abc import *

m = 0
#mass of the particle in GeV
alpha = 0.1
Mp = 2.435*(10**18) #reduced planck mass in GeV
eps = 10**(-3)
t0 = 0 #initial time
GeV_in_g     = 1.782661907e-24
r0 = 10**7
Ti = 10**(-1)
Mi = 10**13*Mp

#Framework

def mesh_r(start, stop, num):
    return(np.linspace(start,stop, num))

def time(t0, tmax, n):
    return(np.linspace(t0,tmax, n))

#Functions

def tau(M0):
    return ((M0)**3)/(3*eps*(Mp**4))

def MBH(M0, t):
    tau_BH = tau(M0)
    return(M0*((1-(t-t0)/tau_BH)**(1/3))*np.heaviside(tau_BH-(t-t0), 0))
    

    
def Th1(M0, t):
    return(1.06*((10**13/GeV_in_g)/(MBH(M0,t)))) #to change later
    #return(10)

def phi1(M0, t):
    return(-(eps/Th(M0,t))*(Mp**4)/(MBH(M0, t)**2)) #diviser par E plutôt que m    
    
def Temp(T0, r):
    Tinf = T0[T0.size - 1]
    r_adj = np.concatenate((r, np.linspace(r[-1], 100*r[-1], 100)))
    T_adj = np.concatenate((T0, Tinf*np.ones(100)))
    return((spip.interp1d(r, T0)))

def g1(T):
    return(100)

g = 100

In [23]:
#Approximate solution functions

y0, y1, y2, y3 = symbols('y0,y1,y2,y3')
z0, z1, z2, z3 = symbols('z0,z1,z2,z3')
y0 = Function('y0')(r,t)
y1 = Function('y1')(r,t)
y2 = Function('y2')(r,t)
#y3 = Function('y3')(r,t)
z0 = Function('z0')(r,t)
z1 = Function('z1')(r,t)
z2 = Function('z2')(r,t)
#z3 = Function('z3')(r,t)

T = Function('T')(r,t)
E = Function('E')(r,t)
phi = Function('phi')(t)
Th = Function('Th')(t)


T0 = Ti
E0 = Th

#Linear differential functions 

L_t = lambda f: diff(f, t)
L_r = lambda f: diff(f, r)
L_rr = lambda f: diff(f,r,r)

#Homotopy equations 

eq_E1 = Eq(L_r(E)+p*(alpha**2 *sqrt(T**3 *E)),0)
eq_T1 = Eq(L_t(T)+p*(-(1/((np.pi*alpha)**2 *g*T)*(45*L_r(T)+r**2*((45/T)*L_r(T)**2 +30*L_rr(T)))+
                    (15*phi/(8*(np.pi*T)**3*g)*L_r(E)))), 0)
           
#Creating approximate solutions
           
T_prime = y0 +p*y1+p**2*y2
E_prime = z0 + p*z1+p**2*z2
eq_E2 = eq_E1.subs(E, E_prime)
eq_E2 = eq_E2.subs(T, T_prime)
eq_T2 = eq_T1.subs(T, T_prime)
eq_T2 = eq_T2.subs(E, E_prime)

#Expanding the above equations for p and derivatives

eq_E3 = expand(eq_E2.doit()) #gives us the canonical form of the polynoms and evaluates it
eq_T3 = expand(eq_T2.doit())

pprint(eq_E3 )
pprint(eq_T3)

#Segregating coefficients of p 

de_E0 = Eq(z0.diff(r), 0) ; icsE0 = {z0.subs(r,0):Th} ;
de_E1 = Eq(eq_E3.lhs.coeff(p**1), 0) ; icsE1 = {z1.subs(r,0):0} ;
de_E2 = Eq(eq_E3.lhs.coeff(p**2), 0) ; icsE2 = {z2.subs(r,0):0} ;
#de_E3 = Eq(eq_E3.lhs.coeff(p**3), 0) ; icsE3 = {z3.subs(r,0):0} ;

de_T0 = Eq(y0.diff(t), 0) ; icsT0 = {y0.subs(t,0): T0} ;
de_T1 = Eq(eq_T3.lhs.coeff(p**1), 0) ; icsT1 = {y1.subs(t,0):0} ;
de_T2 = Eq(eq_T3.lhs.coeff(p**2), 0) ; icsT2 = {y2.subs(t,0):0} ;
#de_T3 = Eq(eq_T3.lhs.coeff(p**3), 0) ; icsT3 = {y3.subs(t,0):0} ; 

#Solutions at given orders
                    
    #for p^0
pdsolve(de_E0)
#
                    

                            __________________________________________________
 2 ∂                       ╱  8   3                     7            2        
p ⋅──(z₂(r, t)) + 0.01⋅p⋅╲╱  p ⋅y₂ (r, t)⋅z₂(r, t) + 3⋅p ⋅y₁(r, t)⋅y₂ (r, t)⋅z
   ∂r                                                                         

______________________________________________________________________________
           7   3                     6            2                     6   2 
₂(r, t) + p ⋅y₂ (r, t)⋅z₁(r, t) + 3⋅p ⋅y₀(r, t)⋅y₂ (r, t)⋅z₂(r, t) + 3⋅p ⋅y₁ (
                                                                              

______________________________________________________________________________
                             6            2                   6   3           
r, t)⋅y₂(r, t)⋅z₂(r, t) + 3⋅p ⋅y₁(r, t)⋅y₂ (r, t)⋅z₁(r, t) + p ⋅y₂ (r, t)⋅z₀(r
                                                                              

_________________________________________________

Eq(z0(r, t), F(-t))

In [24]:
pdsolve(de_T0)

Eq(y0(r, t), F(r))

In [25]:
de_E1 = de_E1.subs(z0, Th)
pdsolve(de_E1)

NotImplementedError: psolve: Cannot solve 0.01*sqrt(p**8*y2(r, t)**3*z2(r, t) + 3*p**7*y1(r, t)*y2(r, t)**2*z2(r, t) + p**7*y2(r, t)**3*z1(r, t) + p**6*Th(t)*y2(r, t)**3 + 3*p**6*y0(r, t)*y2(r, t)**2*z2(r, t) + 3*p**6*y1(r, t)**2*y2(r, t)*z2(r, t) + 3*p**6*y1(r, t)*y2(r, t)**2*z1(r, t) + 3*p**5*Th(t)*y1(r, t)*y2(r, t)**2 + 6*p**5*y0(r, t)*y1(r, t)*y2(r, t)*z2(r, t) + 3*p**5*y0(r, t)*y2(r, t)**2*z1(r, t) + p**5*y1(r, t)**3*z2(r, t) + 3*p**5*y1(r, t)**2*y2(r, t)*z1(r, t) + 3*p**4*Th(t)*y0(r, t)*y2(r, t)**2 + 3*p**4*Th(t)*y1(r, t)**2*y2(r, t) + 3*p**4*y0(r, t)**2*y2(r, t)*z2(r, t) + 3*p**4*y0(r, t)*y1(r, t)**2*z2(r, t) + 6*p**4*y0(r, t)*y1(r, t)*y2(r, t)*z1(r, t) + p**4*y1(r, t)**3*z1(r, t) + 6*p**3*Th(t)*y0(r, t)*y1(r, t)*y2(r, t) + p**3*Th(t)*y1(r, t)**3 + 3*p**3*y0(r, t)**2*y1(r, t)*z2(r, t) + 3*p**3*y0(r, t)**2*y2(r, t)*z1(r, t) + 3*p**3*y0(r, t)*y1(r, t)**2*z1(r, t) + 3*p**2*Th(t)*y0(r, t)**2*y2(r, t) + 3*p**2*Th(t)*y0(r, t)*y1(r, t)**2 + p**2*y0(r, t)**3*z2(r, t) + 3*p**2*y0(r, t)**